In [1]:
import anndata as ad
import pandas as pd
import numpy as np
import scipy.sparse as sp

# Single Cell Sequencing Data

Single cell RNA-SEQ atlas filtered for sporadic amyotrophic lateral sclerosis (sALS) in the Motor Cortex.

In [2]:
# Path to the ANNDATA object
anndata_path = "/home/amrinder/Documents/Github/helical_challenge/counts_combined_filtered_BA4_sALS_PN.h5ad"

# Load the ANNDATA object and keep remove the training data
anndata = ad.read_h5ad(anndata_path)

# In Silico Perturbation Workflow

Perturbing genes belonging to specific cell types.

In [8]:
# Select subset of anndata to target for perturbations
Cell = 'Astro'
new_anndata = anndata[anndata.obs['CellType'] == Cell]
new_anndata = new_anndata[new_anndata.obs['split'] == 'test']

# Condition PN = Non-Pathalogical, ALS = Pathological for ALS
Condition = 'PN'
if Condition:
    new_anndata = new_anndata[new_anndata.obs['Condition'] == Condition]

new_anndata = new_anndata.copy()

Inputs for perturbation workflow

In [9]:
# Genes to perturb dict { Ensemble ID (STR) : Scale (FLOAT), ... }
perturb_genes = {'ENSG00000107317' : 0.05, 'ENSG00000066032' : 4}

# Name for metadata in anndata object (STR)
metadata = 'Genes_and_Scale'

Perturbing selected genes by scale

In [10]:
# Counts matrix
X = new_anndata.X

# Matrix for scaling target genes be specific factor
scale = np.ones(new_anndata.n_vars, dtype=float)

for g, s in perturb_genes.items():
    g_idx = pd.Index(new_anndata.var['ENSID']).get_loc(g)
    scale[g_idx] = float(s)

scale = sp.diags(scale)

# Apply scaling to counts matrix, generates a float64 object
Y = X @ scale
# Round and convert back to int32
np.rint(Y.data, out=Y.data)
Y.data = Y.data.astype(np.int32, copy=False)
Y.eliminate_zeros()

# Store in new_anndata object with associated meta data file
new_anndata.X = Y
new_anndata.uns[metadata] = perturb_genes

Saves anndata file

In [11]:
new_anndata_path = f"/home/amrinder/Documents/Github/helical_challenge/counts_{Cell}_filtered_BA4_{Condition}_PTGDS_CTNNA2.h5ad"
new_anndata.write_h5ad(new_anndata_path)